# Filters

An example of creating a filter:

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

num_taps = 51 # it helps to use an odd number of taps, Type 1 FIR
cut_off = 3000 # Hz
sample_rate = 32000 # Hz

# create our low pass filter
# Hamming window is default
h = signal.firwin(num_taps, cut_off, fs=sample_rate/2)

# plot the impulse response
fig, axs = plt.subplots(2)
axs[0].grid(True)
axs[0].plot(h, '.-')

# Plot the frequency response
H = np.fft.fftshift(np.fft.fft(h, 1024))
H = np.abs(H)
f = np.linspace(-sample_rate/2, sample_rate/2, len(H)) # x axis
axs[1].grid(True)
axs[1].plot(f, np.abs(H), '-')
plt.show()
plt.grid(True)
w = np.linspace(0, 0.5, len(H)//2)
x = H[(len(H)//2):]
plt.plot(w, 20*np.log10(x), '-')
plt.show()